# Pairwise distance analysis with Plink

In [ ]:
## Activate R in jupyter notebook
%load_ext rpy2.ipython

You have already seen one way to visualise genotype data in low dimensions, using smartPCA. We now turn to another method, called MDS and implemented in the software package [Plink](https://zzz.bwh.harvard.edu/plink/).

We will first have to convert the EIGENSTRAT-formatted genotype data that we produced via `trident forge` to Plink format. We can do that using `trident genoconvert`. Open a terminal window in jupyter (up on the left, the button with the plus sign, and then Terminal), and make sure you're in the right environment (e.g. run `conda activate PCA_and_friends`). Then switch to the `session_2` directory via `cd session_2` and then:


In [ ]:
!trident genoconvert -d scratch/forged_package --outFormat PLINK


We can now find the PLINK data in the same directory as the EIGENSTRAT data. Check it out by inspecting the file browser in Jupyter, or by running in the terminal:


In [ ]:
!ls scratch/forged_package/

Where the `.bed`, `.bim` and `.fam` files are used in PLINK. In fact, they are automatically used whenever you use the option `--bfile forged_package/PCA_package` in plink. The first command we're gonna use is for computing pairwise distances of all individuals:


In [ ]:
!plink --bfile scratch/forged_package/PCA_package_1 --distance-matrix --out scratch/pairwise_distances

Now we can do stuff in R. Let's first load the individuals.

In [ ]:
%%R
inds <- readr::read_tsv("scratch/pairwise_distances.mdist.id", col_types="cc", col_names=c("Population", "Individual"))
inds

In [ ]:
%%R
dist_mat <- matrix(scan("scratch/pairwise_distances.mdist"), ncol=1666)
dim(dist_mat)

We can play a bit with trying to visualizing that massive 1666x1666 matrix, for example using the function `heatmap`:

In [ ]:
%%R
?heatmap

Let's first try and filter for a few populations:

In [ ]:
%%R
unique(inds$Population)

In [ ]:
%%R
indices <- inds$Population %in% c('French', 'Greek', 'Nganasan')
head(indices, 40)

In [ ]:
%%R
heatmap(dist_mat[indices,indices], labRow = inds$Population[indices], labCol = inds$Population[indices])


We need to load two libraries:

In [ ]:
%%R
library(ggplot2)
library(magrittr) # This is for the pipe operator %>%
mds_coords <- cmdscale(dist_mat)
colnames(mds_coords) <- c("C1", "C2")
mds_coords <- tibble::as_tibble(mds_coords) %>%
    dplyr::bind_cols(inds)
mds_coords

In [ ]:
%%R
ggplot(mds_coords) + 
    geom_point(aes(x=C1, y=C2)) +
    theme_minimal() +
    coord_equal()

We can then reorient these coordinates like we did the eigenvectors before.

In [ ]:
%%R
corner_inds <- mds_coords %>% dplyr::select(Individual, C1, C2) %>% dplyr::filter(Individual %in% c("HGDP00607", "Sir50"))
if (corner_inds$C1[1] > corner_inds$C1[2]) { mds_coords <- mds_coords %>% mutate(C1=-C1)}
if (corner_inds$C2[1] > corner_inds$C2[2]) { mds_coords <- mds_coords %>% mutate(C2=-C2)}

ggplot(mds_coords) + 
    geom_point(aes(x=C1, y=C2)) +
    theme_minimal() +
    coord_equal()

How does MDS compare to PCA if we restrict to the populations in `poplist1`?

In [ ]:
%%R

## Read in the poplist
poplist1 <- readr::read_tsv("PCA_poplists/PCA_poplist1.txt", col_names = "Pops", col_types = 'c')

## Filter distance matrix
indices_pl1 <- inds$Population %in% poplist1$Pops
dist_mat[indices_pl1, indices_pl1]

## Do MDS
mds_coords_pl1 <- cmdscale(dist_mat[indices_pl1,indices_pl1])
colnames(mds_coords_pl1) <- c("C1", "C2")
mds_coords_pl1 <- tibble::as_tibble(mds_coords_pl1) %>%
    dplyr::bind_cols(inds %>% dplyr::filter(inds$Population %in% poplist1$Pops))
mds_coords_pl1


## Reorient
corner_inds_mds1 <- mds_coords_pl1 %>% dplyr::select(Individual, C1, C2) %>% dplyr::filter(Individual %in% c("HGDP00607", "Sir50"))
if (corner_inds_mds1$C1[1] > corner_inds_mds1$C1[2]) { mds_coords_pl1 <- mds_coords_pl1 %>% mutate(C1=-C1)}
if (corner_inds_mds1$C2[1] > corner_inds_mds1$C2[2]) { mds_coords_pl1 <- mds_coords_pl1 %>% mutate(C2=-C2)}

## Plot
ggplot(mds_coords_pl1) + 
    geom_point(aes(x=C1, y=C2)) +
    theme_minimal() +
    coord_equal()